In [ ]:
# =============================================================================
# Author: Deepthi Kuttichiara and Pubudu Sanjeewani Thihagoda Gamage
# Date: 20-08-2024
# Title: A Novel Non-Iterative Training Method for CNN Classifiers Using Gram-Schmidt Process
# License: MIT License
# =============================================================================

# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping
from tensorflow.keras.applications import VGG16, VGG19, ResNet50, DenseNet121, DenseNet169
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns
from keras.optimizers import SGD,Adam
from keras.callbacks import ReduceLROnPlateau # Learning rate scheduler 

# Set seeds for reproducibility
seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)

# Load CIFAR-10 dataset and normalize
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
y_test_check = y_test

x_train = (x_train.astype('float32')) / 255
x_test = (x_test.astype('float32')) / 255

# One-hot encode labels
num_classes = 10
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

# Load pretrained VGG16 model (without top classification layer)
base_model = VGG16(include_top=False, weights='imagenet', input_shape=(32, 32, 3), classes=y_train.shape[1]) # Choose one of the models: VGG16, VGG19, ResNet50, DenseNet121, or DenseNet169

# Construct full model by adding custom classification layers
model_1 = Sequential()
model_1.add(base_model)
model_1.add(Flatten()

# Add fully connected layers and dropout
model_1.add(Dense(4000,activation=('relu'),input_dim=512))
model_1.add(Dense(2000,activation=('relu'))) 
model_1.add(Dropout(.4))
model_1.add(Dense(1000,activation=('relu'))) 
model_1.add(Dropout(.3))
model_1.add(Dense(500,activation=('relu')))
model_1.add(Dropout(.2))
model_1.add(Dense(10,activation=('softmax')))

# Print model summary
model_1.summary()

# Define learning rate reduction strategy
lrr= ReduceLROnPlateau(
                       monitor='val_accuracy',
                       factor=.01, 
                       patience=3,  
                       min_lr=1e-5) 

# Define training parameters
batch_size= 100
epochs=50
learn_rate=.001

# Define optimizers
sgd=SGD(learning_rate=learn_rate,momentum=.9,nesterov=False)
adam=Adam(learning_rate=learn_rate, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

# Compile the model
model_1.compile(optimizer=sgd,loss='categorical_crossentropy',metrics=['accuracy'])

# Setup data augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
)
datagen.fit(x_train, seed=seed)

# Train the model with data augmentation
history = model_1.fit(datagen.flow(x_train, y_train, batch_size=batch_size, seed=seed),
                    steps_per_epoch=x_train.shape[0] // batch_size,
                    epochs=50,
                    callbacks=[lrr],
                    verbose=1,
                    validation_data=(x_test, y_test))

# Save the trained model
model_1.save("cifar10_vgg16.h5")

# Evaluate on training data
train_loss, train_accuracy = model_1.evaluate(x_train, y_train, verbose=0)
print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")

# Evaluate on test data
test_loss, test_accuracy = model_1.evaluate(x_test, y_test, verbose=0)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)                   │ (None, 1, 1, 512)           │      14,714,688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 512)                 │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 14,714,688 (56.13 MB)

 Non-trainable params: 0 (0.00 B)

...

500/500 ━━━━━━━━━━━━━━━━━━━━ 1706s 3s/step - accuracy: 0.3182 - loss: 1.8425 - val_accuracy: 0.7111 - val_loss: 0.8347 - learning_rate: 0.0010
Epoch 2/50


C:\Users\sanje\anaconda3\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


500/500 ━━━━━━━━━━━━━━━━━━━━ 101s 202ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.7111 - val_loss: 0.8347 - learning_rate: 0.0010
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1839s 4s/step - accuracy: 0.6820 - loss: 0.9294 - val_accuracy: 0.7437 - val_loss: 0.7443 - learning_rate: 0.0010
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 114s 227ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.7437 - val_loss: 0.7443 - learning_rate: 0.0010
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 2026s 4s/step - accuracy: 0.7409 - loss: 0.7579 - val_accuracy: 0.7763 - val_loss: 0.6739 - learning_rate: 0.0010
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 114s 228ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.7763 - val_loss: 0.6739 - learning_rate: 0.0010
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1460s 3s/step - accuracy: 0.7782 - loss: 0.6538 - val_accuracy: 0.7937 - val_loss: 0.6285 - learning_rate: 0.0010
Epoch 8/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 110s 220ms/step -

500/500 ━━━━━━━━━━━━━━━━━━━━ 48s 96ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.8568 - val_loss: 0.4342 - learning_rate: 1.0000e-05
Epoch 45/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 831s 2s/step - accuracy: 0.8816 - loss: 0.3509 - val_accuracy: 0.8578 - val_loss: 0.4324 - learning_rate: 1.0000e-05
Epoch 46/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 48s 95ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.8578 - val_loss: 0.4324 - learning_rate: 1.0000e-05
Epoch 47/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 835s 2s/step - accuracy: 0.8794 - loss: 0.3535 - val_accuracy: 0.8572 - val_loss: 0.4343 - learning_rate: 1.0000e-05
Epoch 48/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 48s 96ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.8572 - val_loss: 0.4343 - learning_rate: 1.0000e-05
Epoch 49/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 833s 2s/step - accuracy: 0.8815 - loss: 0.3524 - val_accuracy: 0.8581 - val_loss: 0.4333 - learning_rate: 1.0000e-05
Epoch 50/50
500/500 ━━━━━━━━━━━━━━━━━

Train Loss: 0.2872, Train Accuracy: 0.9035
Test Loss: 0.4333, Test Accuracy: 0.8581
